## Extremely Randomised Trees

Train ERT (intrinsic and dust attenuated)

In [1]:
import numpy as np
import pickle as pcl

from methods.training import load_data

from sklearn.tree import ExtraTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=False)

Using TensorFlow backend.


In [2]:
from predict import predict

si = predict(fname='data/full_histories_illustris.h5')
si.training_mask()
si.filename

'data/full_histories_illustris.h5'

In [20]:
key = 'Dust'
si.generate_standardisation(key)
features = si.prepare_features(key=key, CNN=True)
predictors = si.load_arr('log_8','SFH')

In [21]:
ERT = ExtraTreeRegressor(criterion='mse')
ERT.get_params()

{'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'random'}

In [23]:
parameters = {
                'min_samples_split': [4,5,6,7,8,10],
                'min_samples_leaf': [7,8,9,10,14],
                'max_leaf_nodes': [130, 150, 160,180, 200, 220, 240]
             }

# parameters = clf.best_params_

$SMAPE = 2 \times \frac{\Sigma \, |Y_{true} \,-\, Y_{pred}|}{\Sigma \, (Y_{true} \,+\, Y_{pred})}$

In [24]:
def custom_SMAPE(y_true, y_pred):
    return np.mean(si._SMAPE(y_true, y_pred))

In [25]:
scorer = make_scorer(custom_SMAPE, greater_is_better=False)

features = np.squeeze(features)

clf = GridSearchCV(ERT, param_grid=parameters, scoring=scorer)
clf.fit(X = features[si.train], y=predictors[si.train])
clf.best_params_

/research/astro/highz/Students/Chris/Learning_SFHs/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'max_leaf_nodes': 240, 'min_samples_leaf': 10, 'min_samples_split': 6}

In [26]:
ERT = clf.best_estimator_
prediction = ERT.predict(features[~si.train])
print("%0.3f percent"%(100 * scorer(ERT, features[~si.train], predictors[~si.train]) * -1))
# print("%0.3f percent"%(scorer(ERT, features[~sp.train], predictors[~sp.train])))

23.050 percent


In [27]:
pcl.dump(clf, open('data/ert_trained_dust.p', 'wb'))